### 네이버 영화 리뷰를 크롤링하여 데이터프레임으로 변환 ###

In [ ]:
import requests
from bs4 import BeautifulSoup 
import time
import csv

In [ ]:
reviews = []
review_data = []
need_reviews_cnt = 1000

for page in range(1,50):
    url = f'https://movie.naver.com/movie/point/af/list.naver?&page={page}'
    html = requests.get(url) #get : request로 url의  html문서의 내용 요청
    soup = BeautifulSoup(html.content,'html.parser') #html을 받아온 문서를 .content로 지정 후 soup객체로 변환 
    reviews = soup.find_all("td",{"class":"title"}) #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    for review in reviews:
        sentence = review.find("a",{"class":"report"}).get("onclick").split("', '")[2] #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
        if sentence != "":
            movie = review.find("a",{"class":"movie color_b"}).get_text()
            score = review.find("em").get_text()
            review_data.append([movie,sentence,int(score)])
            need_reviews_cnt-= 1
        #현재까지 수집된 리뷰가 목표 수집 리뷰보다 많아진 경우 크롤링 중지 
        if need_reviews_cnt < 0:                                  
          break
        time.sleep(0.5) #다음 페이지를 조회하기 전 0.5초 시간 차를 두기

In [ ]:
review_data

[['본즈 앤 올', '잘 봤어요', 9],
 ['올빼미',
  '작품을 구성한 상상력도 뛰어나고 배우들의 연기도 일품. 유해진배우의 광기어린 연기는 처음보는 듯 한데 너무 무섭고요..무섭다고요..',
  7],
 ['스트레인지 월드',
  '동성애가 뜬금없다고는 하지만 거슬릴만큼의 설정도 아님, 그냥 가볍게 지나치면 될 정도인데 이악물고 두들겨패고 보는건 걍 디즈니 망하라고 고사지내는건가.. 그리고 작품 전체를 관통하는 주제를 생각하면 못나올 설정도 아님. 다만 이 작품의 문제는 캐릭터들의 매력이 너무나도 떨어져서 대중들의 관심을 못끈다는것...',
  6],
 ['올빼미', '엔딩 크레딧 다 보고 나왔다. ', 8]]

In [ ]:
columns_name = ["movie","sentence","score"]
with open ( "samples.csv", "w", newline ="",encoding = 'utf8' ) as f:
    write = csv.writer(f)
    write.writerow(columns_name)
    write.writerows(review_data)

In [ ]:
import pandas as pd
data = pd.read_csv("samples.csv")
data

,movie,sentence,score
0,탄생,"전기, 역사영화로서 묵묵히 한 길만 팠기에 매우 만족스러운 작품, 영화적 편집이나 ...",8
1,압꾸정,"영화보고 재미없다는말 잘안하는데,,,,시간이랑 돈을 땅바닥에 갖다 버리고왔네요..마...",1
2,원피스 필름 레드,왜 12세 관람가인줄 알겠다 음악에 돈을 다 넣은걸까 3D도 어설프게 구현해서 전체...,3
3,블랙 팬서: 와칸다 포에버,언니들이 다해먹어서 좋다,10
4,원피스 필름 레드,항상 챙겨보는 만화입니다 재밌게 잘봤어요~,8
...,...,...,...
439,압꾸정,저는 너무 웃겼는데 같이 본 친구들은 다 노잼이래요. 코드가 맞아야 하나봐요.개그코...,10
440,올빼미,몰입하며 봤어요연출력 좋고...연기 좋고사극 스릴러 ..잘 봤습니다추천해요!!,9
441,"오늘 밤, 세계에서 이 사랑이 사라진다 해도",생각이상으로 슬프고 감성적인 내용이 깊게 느껴졌다,10
442,중경삼림,캘리포니아 드림은 기억에 선명하게 남지만 스토리는 정말 취향에 안 맞음 여자주인공 ...,6


In [ ]:
#영화별로 그룹을 지어 평점 평균 구하기
result = data.groupby('movie')['score'].mean()
result

movie
007 두번 산다           5.00
8월의 크리스마스          10.00
가버나움               10.00
가장 보통의 연애          10.00
강릉                  2.00
                   ...  
핸콕                 10.00
허슬                  6.00
헌트                  8.50
헤어질 결심              7.75
헤이지니&럭키강이 비밀의 문    10.00
Name: score, Length: 141, dtype: float64

In [ ]:
type(result)

pandas.core.series.Series

In [ ]:
#평점 상위 20등까지 출력
result = result.sort_values(ascending=False)
result

for i in range(20):
  print(f'{i+1} : {result.index[i]}, {result[i]}점')

1 : 헤이지니&럭키강이 비밀의 문, 10.0점
2 : 데이 시프트, 10.0점
3 : 우주 전쟁, 10.0점
4 : 와일드라이프, 10.0점
5 : 데블스 에드버킷, 10.0점
6 : 여고괴담, 10.0점
7 : 슬리퍼스, 10.0점
8 : 킹덤2: 아득한 대지로, 10.0점
9 : 엔시티 드림 더 무비 : 인 어 드림, 10.0점
10 : 에놀라 홈즈, 10.0점
11 : 언터처블: 1%의 우정, 10.0점
12 : 양성인간, 10.0점
13 : 디모: 벚꽃의 소리, 10.0점
14 : 알 포인트, 10.0점
15 : 8월의 크리스마스, 10.0점
16 : 담보, 10.0점
17 : 악마를 보았다, 10.0점
18 : 마녀(魔女) Part2. The Other One, 10.0점
19 : 아이즈 와이드 셧, 10.0점
20 : 아벤고 공수 군단, 10.0점


In [ ]:
#평점 7~8점 출력
result = result.sort_values(ascending=False)

for i in range(len(result)):
  if 0.7 <= result[i] and result[i] <= 8.0:
    print(f'{i+1} : {result.index[i]} : {result}점')

74 : 아포칼립토 : movie
헤이지니&럭키강이 비밀의 문    10.0
당신이 잠든 사이에         10.0
다크 나이트             10.0
죽지않는 인간들의 밤        10.0
가버나움               10.0
                   ... 
스크루지: 크리스마스 캐럴      1.0
상류사회                1.0
이웃집 토토로             1.0
사잇소리                1.0
이제 그만 끝낼까 해         1.0
Name: score, Length: 141, dtype: float64점
75 : 듄 : movie
헤이지니&럭키강이 비밀의 문    10.0
당신이 잠든 사이에         10.0
다크 나이트             10.0
죽지않는 인간들의 밤        10.0
가버나움               10.0
                   ... 
스크루지: 크리스마스 캐럴      1.0
상류사회                1.0
이웃집 토토로             1.0
사잇소리                1.0
이제 그만 끝낼까 해         1.0
Name: score, Length: 141, dtype: float64점
76 : 만인의 연인 : movie
헤이지니&럭키강이 비밀의 문    10.0
당신이 잠든 사이에         10.0
다크 나이트             10.0
죽지않는 인간들의 밤        10.0
가버나움               10.0
                   ... 
스크루지: 크리스마스 캐럴      1.0
상류사회                1.0
이웃집 토토로             1.0
사잇소리                1.0
이제 그만 끝낼까 해         1.0
Name: score, Length: 141, dtype: float64점
77 : 외계+인 1부 : movie
헤이지니&럭키

In [ ]:
# 나눔폰트 설치
!sudo apt update && sudo apt upgrade -y
!sudo aptitude upgrade -y
!sudo apt autoremove -y
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,056 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRel

In [ ]:
#한글 형태소 분석기인 konlpy를 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 596 kB/s 
     |████████████████████████████████| 465 kB 78.8 MB/s 


In [ ]:
from konlpy.tag import Okt
from wordcloud import WordCloud
from collections import Counter

# text 파일 읽어오기
movieTxt = open('samples.csv', 'rt', encoding='utf-8').read()
print(movieTxt)

movie,sentence,score
탄생,"전기, 역사영화로서 묵묵히 한 길만 팠기에 매우 만족스러운 작품, 영화적 편집이나 재미는 떨어지고 비신자들에게는 지루할수도 있겠지만 김대건 이라는 한 인물의 인생길 및 한국 카톨릭의 고난의 역사가 와닿는 작품. 자산어보를 보고 정씨 형제의 죽음과 유배의 역사적 배경이 궁금한 분들이라면 강추입니다. 윤시윤 및 다양한 배우들을 만나는 즐거움도 있고 ",8
압꾸정,"영화보고 재미없다는말 잘안하는데,,,,시간이랑 돈을 땅바닥에 갖다 버리고왔네요..마블리 보러갔다가 대..실망 ",1
원피스 필름 레드,왜 12세 관람가인줄 알겠다 음악에 돈을 다 넣은걸까 3D도 어설프게 구현해서 전체적으로 비어보이고 작화에 질도 떨어지고 내가 알던 원피스가 맞나 싶다  필름 골드에서 몇배나 퇴보한느낌,3
블랙 팬서: 와칸다 포에버,언니들이 다해먹어서 좋다,10
원피스 필름 레드,항상 챙겨보는 만화입니다 재밌게 잘봤어요~,8
올빼미,영화 자체적으론 재밌었지만 어떤 한 장면이 징그럽고 계속 생각날 수 있으니 시청 주의 하세요 나 그거땜에 밤에 잠 혼자 못 잠,8
"오늘 밤, 세계에서 이 사랑이 사라진다 해도",웃참 챌린지했음.. 갑자기 급발진하는 장면이 너무 많이 나오고 개연성도.. 뻔한 클리셰까지 완벽!,1
올빼미,영화 보면서 이렇게 심장이 뛰는 건 처음이였어요... 제발 한번 만 보세요 제발 배우님 얼굴도 열일합니다...♥,10
놉,아는만큼 보이는,10
헌트,잘 만든데뷔작,10
본즈 앤 올,잘 봤어요,9
올빼미,작품을 구성한 상상력도 뛰어나고 배우들의 연기도 일품. 유해진배우의 광기어린 연기는 처음보는 듯 한데 너무 무섭고요..무섭다고요..,7
스트레인지 월드,"동성애가 뜬금없다고는 하지만 거슬릴만큼의 설정도 아님, 그냥 가볍게 지나치면 될 정도인데 이악물고 두들겨패고 보는건 걍 디즈니 망하라고 고사지내는건가.. 그리고 작품 전체를 관통하는 주제를 생각하면 못나올 설정도 아님. 다만 이 작품의 문제는 캐릭터들의 매력이 너무나도 떨어져서 대중들